In [1]:
from pytorch_lightning import Trainer
from pytorch_lightning.logging import TestTubeLogger
# from pytorch_lightning.callbacks import ModelCheckpoint
from trainer import VAETrainer
from models import VanillaVAE
import torch
from torchvision.datasets import CelebA
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms
from fastprogress.fastprogress import master_bar, progress_bar

In [2]:
EXP_NAME = "VanillaVAE"
SAVE_DIR="logs/"
DATA_DIR = "../../shared/Data/"
BATCH_SIZE = 32
NUM_EPOCHS = 10
LR = 1e-3
DEVICE = 'cuda:2'
torch.manual_seed = 135

In [3]:
SetRange = transforms.Lambda(lambda X: 2 * X - 1.)
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.CenterCrop(148),
                                transforms.Resize(64),
                                transforms.ToTensor(),
                                SetRange])

train_loader = DataLoader(CelebA(root = DATA_DIR,
                                 split = "train",
                                 transform=transform,
                                 download=True),
                          batch_size= BATCH_SIZE,
                          shuffle = True,
                          drop_last=True)

val_loader = DataLoader(CelebA(root = DATA_DIR,
                                 split = "test",
                                 transform=transform,
                                 download=True),
                          batch_size= BATCH_SIZE,
                          shuffle = True,
                          drop_last=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = VanillaVAE(3, latent_dim=128)

optimizer = optim.Adam(model.parameters(), lr=LR)

In [6]:
model.train()
model = model.to(DEVICE)

mb = master_bar(range(NUM_EPOCHS))
for epoch in mb:
    for idx, batch in enumerate(progress_bar(train_loader, parent=mb)):
        optimizer.zero_grad()
        real_img, _ = batch
        real_img = real_img. to(DEVICE)

        recons_img, mu, log_var = model(real_img)
        loss = model.loss_function(recons_img, real_img, mu, log_var)
        mb.child.comment = f'Recons Loss:{loss}'
        loss.backward()
        
        optimizer.step()
    

KeyboardInterrupt: 

In [8]:
from pytorch_lightning.logging.t import TensorBoardLogger

ImportError: cannot import name 'TensorBoardLogger'